In [2]:
### Reloads modules properly
%load_ext autoreload
%autoreload 2

In [3]:
import os
os.sys.path.append('../')

In [4]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import mplhep as hep
hep.style.use('CMS')
import glob

# Understanding how to correctly classify the tracks associated to a vertex

In [5]:
!ls /mnt/storage/lborgna/track/l1_nnt/OldKF_TTbar_170K_quality.root

/mnt/storage/lborgna/track/l1_nnt/OldKF_TTbar_170K_quality.root


In [6]:
import uproot

In [7]:
f = uproot.open('/mnt/storage/lborgna/track/l1_nnt/OldKF_TTbar_170K_quality.root')

In [8]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:            62G        2.5G         53G         32M        6.8G         59G
Swap:           63G         44G         19G


In [9]:
t = f['L1TrackNtuple/eventTree;1']

In [10]:
t.keys()

['trk_pt',
 'trk_eta',
 'trk_phi',
 'trk_d0',
 'trk_z0',
 'trk_chi2',
 'trk_chi2dof',
 'trk_chi2rphi',
 'trk_chi2rz',
 'trk_bendchi2',
 'trk_nstub',
 'trk_lhits',
 'trk_dhits',
 'trk_seed',
 'trk_hitpattern',
 'trk_phiSector',
 'trk_genuine',
 'trk_loose',
 'trk_unknown',
 'trk_combinatoric',
 'trk_fake',
 'trk_MVA1',
 'trk_MVA2',
 'trk_matchtp_pdgid',
 'trk_matchtp_pt',
 'trk_matchtp_eta',
 'trk_matchtp_phi',
 'trk_matchtp_z0',
 'trk_matchtp_dxy',
 'KFtrk_inv2R',
 'KFtrk_cot',
 'KFtrk_zT',
 'KFtrk_phiT',
 'KFtrk_match',
 'KFtrk_etasector',
 'KFtrk_phisector',
 'KFtrk_r1',
 'KFtrk_phi1',
 'KFtrk_z1',
 'KFtrk_dPhi1',
 'KFtrk_dZ1',
 'KFtrk_layer1',
 'KFtrk_r2',
 'KFtrk_phi2',
 'KFtrk_z2',
 'KFtrk_dPhi2',
 'KFtrk_dZ2',
 'KFtrk_layer2',
 'KFtrk_r3',
 'KFtrk_phi3',
 'KFtrk_z3',
 'KFtrk_dPhi3',
 'KFtrk_dZ3',
 'KFtrk_layer3',
 'KFtrk_r4',
 'KFtrk_phi4',
 'KFtrk_z4',
 'KFtrk_dPhi4',
 'KFtrk_dZ4',
 'KFtrk_layer4',
 'tp_pt',
 'tp_eta',
 'tp_phi',
 'tp_dxy',
 'tp_d0',
 'tp_z0',
 'tp_d0_prod',
 't

In [11]:
columns=['trk_pt',
 'trk_eta',
 'trk_phi',
 'trk_d0',
 'trk_z0',
 'trk_fake',
 'tp_pt',
 'tp_eta',
 'tp_phi',
 'tp_dxy',
 'tp_d0',
 'tp_z0',
 'tp_eventid',
 'pv_MC']
 

In [12]:
dfs = t.arrays(columns, library='pd')

In [13]:
dfs[0].head()

trk_pt   trk_eta   trk_phi  trk_d0    trk_z0  trk_fake
entry subentry                                                          
0     0         2.623967  1.301414 -0.289234   999.0 -0.878906         2
      1         3.435026  0.566814 -0.159267   999.0  3.339844         2
      2         3.774908  1.815973 -0.240375   999.0  4.921875         1
      3         2.641448 -1.435747 -0.128747   999.0 -0.468750         2
      4         2.320653  1.837421  0.032604   999.0  4.980469         1

In [14]:
dfs[1].head()

tp_pt    tp_eta    tp_phi   tp_dxy     tp_d0     tp_z0  \
entry subentry                                                               
0     0          2.139707 -0.042819 -2.249776  0.00074 -0.000188  5.055195   
      1          3.263992 -0.297909 -2.668668  0.00074  0.000119  5.055390   
      2          2.888089  1.501334 -0.832884  0.00074 -0.000731  5.055292   
      3          3.574382 -0.151156  2.117383  0.00074  0.000738  5.055151   
      4         18.492512 -0.111532 -1.921995  0.00074 -0.000417  5.055228   

                tp_eventid  
entry subentry              
0     0                  0  
      1                  0  
      2                  0  
      3                  0  
      4                  0

In [15]:
dfs[2].head()

,,pv_MC
entry,subentry,
0,0,5.055163
1,0,-3.608661
2,0,-0.261453
3,0,2.803285
4,0,-1.400667


In [16]:
trk = dfs[0]
TP = dfs[1]
mc = dfs[2]

In [17]:
trk.groupby(level=0)['trk_fake'].value_counts()

entry   trk_fake
0       2           144
        1            15
1       2           124
        1            13
        0             2
                   ... 
222974  1            37
        0             2
222975  2           182
        1            24
        0            10
Name: trk_fake, Length: 652067, dtype: int64

In [19]:
trk['is_pv'] = trk['trk_fake'].apply(lambda x: 1 if x==1 else 0)

In [20]:
trk

trk_pt   trk_eta   trk_phi  trk_d0    trk_z0  trk_fake  \
entry  subentry                                                             
0      0         2.623967  1.301414 -0.289234   999.0 -0.878906         2   
       1         3.435026  0.566814 -0.159267   999.0  3.339844         2   
       2         3.774908  1.815973 -0.240375   999.0  4.921875         1   
       3         2.641448 -1.435747 -0.128747   999.0 -0.468750         2   
       4         2.320653  1.837421  0.032604   999.0  4.980469         1   
...                   ...       ...       ...     ...       ...       ...   
222975 211       2.044568 -2.407456 -0.857553   999.0  1.113281         2   
       212       2.491473 -2.141478 -0.616329   999.0  8.847656         2   
       213       2.669393  2.230316 -0.748241   999.0 -0.234375         2   
       214       3.767222 -1.994298 -0.753691   999.0 -4.746094         2   
       215       1.988538 -2.052655 -0.790385   999.0  0.996094         2   

                 is_pv  
entry  subentry         
0      0             0  
       1             0  
       2             1  
       3             0  
       4             1  
...                ...  
222975 211           0  
       212           0  
       213           0  
       214           0  
       215           0  

[40780722 rows x 7 columns]

In [21]:
def fast_histo(z0: np.array, pt: np.array, bin_edges: np.array) -> pd.Series:
    "Event-level Fast Histo implementation to return the reconstructed primary vertex"

    histo = np.histogram(z0, bins=bin_edges, weights=pt)[0]
    
    labels = np.zeros(len(z0), dtype=int)

    max_idx = np.argmax(histo)

    lower_bin_bound = bin_edges[max_idx]
    upper_bin_bound = bin_edges[max_idx + 1]

    in_max_bin_mask = (z0 > lower_bin_bound) & (z0 < upper_bin_bound)
    labels[in_max_bin_mask] = 1

#     z0_reco = z0[in_max_bin_mask].mean()

    return labels




In [95]:
# bin_edges = np.arange(-15, 15+0.3, 0.3)
bin_edges = np.linspace(-15, 15, 256)

In [96]:
test = trk.groupby(level=0).apply(
    lambda x: fast_histo(x["trk_z0"], x["trk_pt"], bin_edges)
)

/mnt/storage/lborgna/condaenv/db-clustering/lib/python3.8/site-packages/numpy/lib/function_base.py:1292: RuntimeWarning: invalid value encountered in subtract
  a = op(a[slice1], a[slice2])


In [97]:
test = pd.DataFrame(test, columns=['FH_label'])

In [98]:
test_exploded = test.explode('FH_label')

In [99]:
test_exploded['FH_label'].values

array([0, 0, 1, ..., 0, 0, 0], dtype=object)

In [100]:
trk['FH_label'] = test_exploded['FH_label'].values

In [101]:
trk

trk_pt   trk_eta   trk_phi  trk_d0    trk_z0  trk_fake  \
entry  subentry                                                             
0      0         2.623967  1.301414 -0.289234   999.0 -0.878906         2   
       1         3.435026  0.566814 -0.159267   999.0  3.339844         2   
       2         3.774908  1.815973 -0.240375   999.0  4.921875         1   
       3         2.641448 -1.435747 -0.128747   999.0 -0.468750         2   
       4         2.320653  1.837421  0.032604   999.0  4.980469         1   
...                   ...       ...       ...     ...       ...       ...   
222975 211       2.044568 -2.407456 -0.857553   999.0  1.113281         2   
       212       2.491473 -2.141478 -0.616329   999.0  8.847656         2   
       213       2.669393  2.230316 -0.748241   999.0 -0.234375         2   
       214       3.767222 -1.994298 -0.753691   999.0 -4.746094         2   
       215       1.988538 -2.052655 -0.790385   999.0  0.996094         2   

                 is_pv FH_label  
entry  subentry                  
0      0             0        0  
       1             0        0  
       2             1        1  
       3             0        0  
       4             1        1  
...                ...      ...  
222975 211           0        0  
       212           0        0  
       213           0        0  
       214           0        0  
       215           0        0  

[40780722 rows x 8 columns]

In [155]:
trk['FH_label'].value_counts()

0    38822193
1     1958529
Name: FH_label, dtype: int64

In [103]:
trk['is_pv'].value_counts()

0    34584802
1     6195920
Name: is_pv, dtype: int64

In [104]:
trk.head()

trk_pt   trk_eta   trk_phi  trk_d0    trk_z0  trk_fake  \
entry subentry                                                             
0     0         2.623967  1.301414 -0.289234   999.0 -0.878906         2   
      1         3.435026  0.566814 -0.159267   999.0  3.339844         2   
      2         3.774908  1.815973 -0.240375   999.0  4.921875         1   
      3         2.641448 -1.435747 -0.128747   999.0 -0.468750         2   
      4         2.320653  1.837421  0.032604   999.0  4.980469         1   

                is_pv FH_label  
entry subentry                  
0     0             0        0  
      1             0        0  
      2             1        1  
      3             0        0  
      4             1        1

In [105]:
from sklearn.metrics import confusion_matrix, roc_auc_score, classification_report

In [106]:
trk["FH_label"] = trk["FH_label"].astype(int)

In [107]:
event1 = trk.iloc[0:159, :]

In [108]:
tn, fp, fn, tp = confusion_matrix(trk["is_pv"].values, trk["FH_label"].values).ravel()

In [109]:
(tn, fp, fn, tp)

(34226915, 357887, 4595278, 1600642)

In [110]:
TPR = tp/(tp+fn)
TPR

0.2583380676316027

In [111]:
FPR = fp/(tn+fp)
FPR

0.010348100301398286

In [113]:

print(
    classification_report(
        trk["is_pv"].values, trk["FH_label"].values, target_names=["not PV", "PV"]
    )
)

              precision    recall  f1-score   support

      not PV       0.88      0.99      0.93  34584802
          PV       0.82      0.26      0.39   6195920

    accuracy                           0.88  40780722
   macro avg       0.85      0.62      0.66  40780722
weighted avg       0.87      0.88      0.85  40780722



In [112]:
roc_auc_score(trk["is_pv"].values, trk["FH_label"].values)

0.6239949836651022

In [149]:
trk

trk_pt   trk_eta   trk_phi  trk_d0    trk_z0  trk_fake  \
entry  subentry                                                             
0      0         2.623967  1.301414 -0.289234   999.0 -0.878906         2   
       1         3.435026  0.566814 -0.159267   999.0  3.339844         2   
       2         3.774908  1.815973 -0.240375   999.0  4.921875         1   
       3         2.641448 -1.435747 -0.128747   999.0 -0.468750         2   
       4         2.320653  1.837421  0.032604   999.0  4.980469         1   
...                   ...       ...       ...     ...       ...       ...   
222975 211       2.044568 -2.407456 -0.857553   999.0  1.113281         2   
       212       2.491473 -2.141478 -0.616329   999.0  8.847656         2   
       213       2.669393  2.230316 -0.748241   999.0 -0.234375         2   
       214       3.767222 -1.994298 -0.753691   999.0 -4.746094         2   
       215       1.988538 -2.052655 -0.790385   999.0  0.996094         2   

                 is_pv  FH_label  
entry  subentry                   
0      0             0         0  
       1             0         0  
       2             1         1  
       3             0         0  
       4             1         1  
...                ...       ...  
222975 211           0         0  
       212           0         0  
       213           0         0  
       214           0         0  
       215           0         0  

[40780722 rows x 8 columns]

In [121]:
fh_z0 = trk.groupby(['entry','FH_label'])['trk_z0'].mean().reset_index()

In [150]:
fh_z0

,entry,FH_label,trk_z0
0,0,0,-0.405044
1,0,1,4.962891
2,1,0,-0.710449
3,1,1,-3.437500
4,2,0,1.941002
...,...,...,...
445919,222973,1,-3.427734
445920,222974,0,-1.394681
445921,222974,1,-4.472656
445922,222975,0,-0.093358


In [125]:
fh_z0_values = fh_z0.loc[fh_z0['FH_label']==1, 'trk_z0']

In [152]:
fh_z0_values.reset_index()

,index,trk_z0
0,1,4.962891
1,3,-3.437500
2,5,-0.265625
3,7,2.732980
4,9,-1.307373
...,...,...
222943,445915,-0.612571
222944,445917,0.839844
222945,445919,-3.427734
222946,445921,-4.472656


In [129]:
mc.shape

(222976, 1)

In [130]:
fh_z0.shape

(445924, 3)

In [136]:
from pv_utils import truth_pv_z0

In [137]:
truth_z0 = truth_pv_z0(TP)

In [147]:
mc

,,pv_MC
entry,subentry,
0,0,5.055163
1,0,-3.608661
2,0,-0.261453
3,0,2.803285
4,0,-1.400667
...,...,...
222971,0,-0.698029
222972,0,0.520282
222973,0,-3.604865


In [146]:
fh_z0_values.reset_index()

,index,trk_z0
0,1,4.962891
1,3,-3.437500
2,5,-0.265625
3,7,2.732980
4,9,-1.307373
...,...,...
222943,445915,-0.612571
222944,445917,0.839844
222945,445919,-3.427734
222946,445921,-4.472656


In [143]:
truth_z0

entry
0         5.054894
1        -3.599295
2        -0.261687
3         2.803424
4        -1.400962
            ...   
222971   -0.698044
222972    0.516307
222973   -3.605887
222974   -4.491471
222975    1.365055
Name: tp_z0, Length: 222941, dtype: float32

In [144]:
residual =truth_z0 - fh_z0_values.values

ValueError: operands could not be broadcast together with shapes (222941,) (222948,) 

In [142]:
residual

0              NaN
1        -8.562186
2              NaN
3         6.240924
4              NaN
            ...   
445915         NaN
445917         NaN
445919         NaN
445921         NaN
445923         NaN
Length: 334428, dtype: float32

# DBSCAN Classification accuracy

In [242]:
event1_pt = event1.trk_pt
event1_z0 = event1.trk_z0

In [76]:
trk

trk_pt   trk_eta   trk_phi  trk_d0    trk_z0  trk_fake  \
entry  subentry                                                             
0      0         2.623967  1.301414 -0.289234   999.0 -0.878906         2   
       1         3.435026  0.566814 -0.159267   999.0  3.339844         2   
       2         3.774908  1.815973 -0.240375   999.0  4.921875         1   
       3         2.641448 -1.435747 -0.128747   999.0 -0.468750         2   
       4         2.320653  1.837421  0.032604   999.0  4.980469         1   
...                   ...       ...       ...     ...       ...       ...   
222975 211       2.044568 -2.407456 -0.857553   999.0  1.113281         2   
       212       2.491473 -2.141478 -0.616329   999.0  8.847656         2   
       213       2.669393  2.230316 -0.748241   999.0 -0.234375         2   
       214       3.767222 -1.994298 -0.753691   999.0 -4.746094         2   
       215       1.988538 -2.052655 -0.790385   999.0  0.996094         2   

                 is_pv  FH_label  
entry  subentry                   
0      0             0         0  
       1             0         0  
       2             1         1  
       3             0         0  
       4             1         1  
...                ...       ...  
222975 211           0         1  
       212           0         0  
       213           0         0  
       214           0         0  
       215           0         1  

[40780722 rows x 8 columns]

In [85]:
from sklearn.cluster import DBSCAN

In [86]:
def pv_dbscan(z0: np.array, eps: float, minPts: int) -> pd.Series:
    "Event-level DBSCAN implementation to return the reconstructed primary vertex"

    db_clustering = DBSCAN(eps=eps, min_samples=minPts).fit(z0.values.reshape(-1, 1))

    return db_clustering.labels_

In [87]:
dbscan_labels = trk.groupby(level=0).apply(lambda x: pv_dbscan(x["trk_z0"], 0.08, 2))

In [156]:
dbscan_labels

entry
0         [0, -1, 1, 2, 1, 2, -1, 3, 4, 5, 0, 6, 5, 0, 7...
1         [0, -1, 1, 2, 3, -1, 4, 5, 1, -1, -1, 6, 0, 0,...
2         [0, 1, 2, -1, 3, 4, 5, 6, 1, 7, 5, 8, 5, 2, 9,...
3         [0, 0, 1, 1, 1, 2, 3, 1, 4, 5, 5, -1, 6, 7, -1...
4         [0, 1, 2, 3, 4, 1, 1, 4, 5, 6, 4, 7, 8, 0, 9, ...
                                ...                        
222971    [-1, 0, 1, 1, 2, 3, 1, 2, 4, -1, 4, 5, 6, -1, ...
222972    [0, -1, 1, 2, 3, 3, 2, 3, -1, -1, 3, 3, 1, -1,...
222973    [0, 1, 2, 3, -1, 4, 5, 6, 7, 8, 3, -1, 8, 9, 1...
222974    [0, 0, 1, 1, -1, 2, 1, 3, 4, 5, 5, 0, 6, -1, 7...
222975    [0, 1, 0, 2, 3, 4, 1, 3, 2, 0, 5, 6, 7, 0, 8, ...
Length: 222976, dtype: object

In [94]:
dbscan_labels_exploded = pd.DataFrame(dbscan_labels,columns=['DBSCAN_labels']).explode("DBSCAN_labels")

In [188]:
trk

trk_pt   trk_eta   trk_phi  trk_d0    trk_z0  trk_fake  \
entry  subentry                                                             
0      0         2.623967  1.301414 -0.289234   999.0 -0.878906         2   
       1         3.435026  0.566814 -0.159267   999.0  3.339844         2   
       2         3.774908  1.815973 -0.240375   999.0  4.921875         1   
       3         2.641448 -1.435747 -0.128747   999.0 -0.468750         2   
       4         2.320653  1.837421  0.032604   999.0  4.980469         1   
...                   ...       ...       ...     ...       ...       ...   
222975 211       2.044568 -2.407456 -0.857553   999.0  1.113281         2   
       212       2.491473 -2.141478 -0.616329   999.0  8.847656         2   
       213       2.669393  2.230316 -0.748241   999.0 -0.234375         2   
       214       3.767222 -1.994298 -0.753691   999.0 -4.746094         2   
       215       1.988538 -2.052655 -0.790385   999.0  0.996094         2   

                 is_pv  FH_label DBSCAN_labels  
entry  subentry                                 
0      0             0         0             0  
       1             0         0            -1  
       2             1         1             1  
       3             0         0             2  
       4             1         1             1  
...                ...       ...           ...  
222975 211           0         0             3  
       212           0         0            19  
       213           0         0            22  
       214           0         0             9  
       215           0         0             3  

[40780722 rows x 9 columns]

In [171]:
test = trk.groupby(['entry', 'DBSCAN_labels'])['trk_pt'].sum()

In [201]:
test

entry   DBSCAN_labels
0       -1               54.307575
         0               32.500057
         1               84.025032
         2               27.201273
         3               11.046275
                           ...    
222975   18               7.006916
         19               8.041750
         20              21.514009
         21               5.532553
         22               7.240000
Name: trk_pt, Length: 6770860, dtype: float32

In [203]:
max_idx = test.reset_index().groupby('entry').idxmax()['trk_pt']

In [206]:
trk

trk_pt   trk_eta   trk_phi  trk_d0    trk_z0  trk_fake  \
entry  subentry                                                             
0      0         2.623967  1.301414 -0.289234   999.0 -0.878906         2   
       1         3.435026  0.566814 -0.159267   999.0  3.339844         2   
       2         3.774908  1.815973 -0.240375   999.0  4.921875         1   
       3         2.641448 -1.435747 -0.128747   999.0 -0.468750         2   
       4         2.320653  1.837421  0.032604   999.0  4.980469         1   
...                   ...       ...       ...     ...       ...       ...   
222975 211       2.044568 -2.407456 -0.857553   999.0  1.113281         2   
       212       2.491473 -2.141478 -0.616329   999.0  8.847656         2   
       213       2.669393  2.230316 -0.748241   999.0 -0.234375         2   
       214       3.767222 -1.994298 -0.753691   999.0 -4.746094         2   
       215       1.988538 -2.052655 -0.790385   999.0  0.996094         2   

                 is_pv  FH_label DBSCAN_labels  
entry  subentry                                 
0      0             0         0             0  
       1             0         0            -1  
       2             1         1             1  
       3             0         0             2  
       4             1         1             1  
...                ...       ...           ...  
222975 211           0         0             3  
       212           0         0            19  
       213           0         0            22  
       214           0         0             9  
       215           0         0             3  

[40780722 rows x 9 columns]

In [210]:
trk.query('entry == 4').groupby('DBSCAN_labels')['trk_pt'].sum()

DBSCAN_labels
-1      73.161888
 0     270.321960
 1      22.027390
 2      23.885500
 3       4.148731
 4      82.821579
 5      13.400341
 6       9.723491
 7      14.803770
 8       5.589791
 9       8.398414
 10      4.997448
 11     43.535095
 12     39.899456
 13      7.610696
 14     26.812929
 15      7.315086
 16     24.878593
 17     11.164557
 18      5.060818
 19      6.234063
 20     12.481120
 21     10.898566
 22      7.830121
 23      6.165878
 24      4.189067
 25      7.628840
Name: trk_pt, dtype: float32

In [208]:
trk.query('entry == 222972').groupby('DBSCAN_labels')['trk_pt'].sum()

DBSCAN_labels
-1     98.283119
 0      8.873867
 1     13.415939
 2     66.578949
 3     31.848261
 4     19.456566
 5     76.437126
 6      8.884796
 7      6.376172
 8     12.590846
 9     14.737280
 10    14.444049
 11    10.606956
 12     6.691581
 13    17.114050
 14     5.814254
 15    20.926205
 16    71.462250
 17    17.631334
 18     4.231463
 19    11.438664
 20     4.169462
 21     5.935084
Name: trk_pt, dtype: float32

In [216]:
pv_labels = test.iloc[max_idx].reset_index()['DBSCAN_labels']

In [238]:
pv_labels.reset_index()

,index,DBSCAN_labels
0,0,1
1,1,7
2,2,5
3,3,1
4,4,0
...,...,...
222971,222971,1
222972,222972,-1
222973,222973,3
222974,222974,1


In [239]:
pv_labels.reset_index()['DBSCAN_labels'].reindex(index=trk.index, method='nearest')

TypeError: '<' not supported between instances of 'tuple' and 'int'

In [232]:
trk.shape

(40780722, 9)

In [226]:
pv_labels.reset_index().reindex_like(trk)

trk_pt  trk_eta  trk_phi  trk_d0  trk_z0  trk_fake  is_pv  \
entry  subentry                                                              
0      0            NaN      NaN      NaN     NaN     NaN       NaN    NaN   
       1            NaN      NaN      NaN     NaN     NaN       NaN    NaN   
       2            NaN      NaN      NaN     NaN     NaN       NaN    NaN   
       3            NaN      NaN      NaN     NaN     NaN       NaN    NaN   
       4            NaN      NaN      NaN     NaN     NaN       NaN    NaN   
...                 ...      ...      ...     ...     ...       ...    ...   
222975 211          NaN      NaN      NaN     NaN     NaN       NaN    NaN   
       212          NaN      NaN      NaN     NaN     NaN       NaN    NaN   
       213          NaN      NaN      NaN     NaN     NaN       NaN    NaN   
       214          NaN      NaN      NaN     NaN     NaN       NaN    NaN   
       215          NaN      NaN      NaN     NaN     NaN       NaN    NaN   

                 FH_label  DBSCAN_labels  
entry  subentry                           
0      0              NaN            NaN  
       1              NaN            NaN  
       2              NaN            NaN  
       3              NaN            NaN  
       4              NaN            NaN  
...                   ...            ...  
222975 211            NaN            NaN  
       212            NaN            NaN  
       213            NaN            NaN  
       214            NaN            NaN  
       215            NaN            NaN  

[40780722 rows x 9 columns]

In [217]:
trk

trk_pt   trk_eta   trk_phi  trk_d0    trk_z0  trk_fake  \
entry  subentry                                                             
0      0         2.623967  1.301414 -0.289234   999.0 -0.878906         2   
       1         3.435026  0.566814 -0.159267   999.0  3.339844         2   
       2         3.774908  1.815973 -0.240375   999.0  4.921875         1   
       3         2.641448 -1.435747 -0.128747   999.0 -0.468750         2   
       4         2.320653  1.837421  0.032604   999.0  4.980469         1   
...                   ...       ...       ...     ...       ...       ...   
222975 211       2.044568 -2.407456 -0.857553   999.0  1.113281         2   
       212       2.491473 -2.141478 -0.616329   999.0  8.847656         2   
       213       2.669393  2.230316 -0.748241   999.0 -0.234375         2   
       214       3.767222 -1.994298 -0.753691   999.0 -4.746094         2   
       215       1.988538 -2.052655 -0.790385   999.0  0.996094         2   

                 is_pv  FH_label DBSCAN_labels  
entry  subentry                                 
0      0             0         0             0  
       1             0         0            -1  
       2             1         1             1  
       3             0         0             2  
       4             1         1             1  
...                ...       ...           ...  
222975 211           0         0             3  
       212           0         0            19  
       213           0         0            22  
       214           0         0             9  
       215           0         0             3  

[40780722 rows x 9 columns]

In [219]:
trk.index

MultiIndex([(     0,   0),
            (     0,   1),
            (     0,   2),
            (     0,   3),
            (     0,   4),
            (     0,   5),
            (     0,   6),
            (     0,   7),
            (     0,   8),
            (     0,   9),
            ...
            (222975, 206),
            (222975, 207),
            (222975, 208),
            (222975, 209),
            (222975, 210),
            (222975, 211),
            (222975, 212),
            (222975, 213),
            (222975, 214),
            (222975, 215)],
           names=['entry', 'subentry'], length=40780722)

In [ ]:
trk.groupby(level=0).apply(lambda x:)

In [218]:
trk.index()

TypeError: 'MultiIndex' object is not callable

In [202]:
test.reset_index().groupby('entry').apply(pd.DataFrame.sort_values, 'trk_pt', ascending=False)

entry  DBSCAN_labels     trk_pt
entry                                           
0      2             0              1  84.025032
       0             0             -1  54.307575
       6             0              5  44.067730
       13            0             12  32.853683
       1             0              0  32.500057
...                ...            ...        ...
222975 6770847  222975             10   5.772072
       6770844  222975              7   5.579736
       6770858  222975             21   5.532553
       6770848  222975             11   4.915463
       6770852  222975             15   4.743252

[6770860 rows x 3 columns]

In [199]:
test.reset_index().groupby(['entry','DBSCAN_labels']).sort_values(by=['trk_pt'],ascending=False)

AttributeError: 'DataFrameGroupBy' object has no attribute 'reset_index'

In [181]:
test.reset_index().groupby(['entry'])['trk_pt'].nlargest(1)

entry          
0       2           84.025032
1       34         104.907082
2       58         226.240372
3       79         301.090637
4       103        270.321960
                      ...    
222971  6770728    256.535797
222972  6770755     98.283119
222973  6770782    174.343292
222974  6770810    259.831726
222975  6770840    344.987152
Name: trk_pt, Length: 222976, dtype: float32

In [153]:
trk['DBSCAN_labels'] = dbscan_labels_exploded['DBSCAN_labels'].values

### trk

In [183]:
trk

trk_pt   trk_eta   trk_phi  trk_d0    trk_z0  trk_fake  \
entry  subentry                                                             
0      0         2.623967  1.301414 -0.289234   999.0 -0.878906         2   
       1         3.435026  0.566814 -0.159267   999.0  3.339844         2   
       2         3.774908  1.815973 -0.240375   999.0  4.921875         1   
       3         2.641448 -1.435747 -0.128747   999.0 -0.468750         2   
       4         2.320653  1.837421  0.032604   999.0  4.980469         1   
...                   ...       ...       ...     ...       ...       ...   
222975 211       2.044568 -2.407456 -0.857553   999.0  1.113281         2   
       212       2.491473 -2.141478 -0.616329   999.0  8.847656         2   
       213       2.669393  2.230316 -0.748241   999.0 -0.234375         2   
       214       3.767222 -1.994298 -0.753691   999.0 -4.746094         2   
       215       1.988538 -2.052655 -0.790385   999.0  0.996094         2   

                 is_pv  FH_label DBSCAN_labels  
entry  subentry                                 
0      0             0         0             0  
       1             0         0            -1  
       2             1         1             1  
       3             0         0             2  
       4             1         1             1  
...                ...       ...           ...  
222975 211           0         0             3  
       212           0         0            19  
       213           0         0            22  
       214           0         0             9  
       215           0         0             3  

[40780722 rows x 9 columns]

In [90]:
len(np.arange(-15, 15+0.3, 0.3))

101

In [88]:
np.linspace(-15, 15, 256)

array([-15.        , -14.88235294, -14.76470588, -14.64705882,
       -14.52941176, -14.41176471, -14.29411765, -14.17647059,
       -14.05882353, -13.94117647, -13.82352941, -13.70588235,
       -13.58823529, -13.47058824, -13.35294118, -13.23529412,
       -13.11764706, -13.        , -12.88235294, -12.76470588,
       -12.64705882, -12.52941176, -12.41176471, -12.29411765,
       -12.17647059, -12.05882353, -11.94117647, -11.82352941,
       -11.70588235, -11.58823529, -11.47058824, -11.35294118,
       -11.23529412, -11.11764706, -11.        , -10.88235294,
       -10.76470588, -10.64705882, -10.52941176, -10.41176471,
       -10.29411765, -10.17647059, -10.05882353,  -9.94117647,
        -9.82352941,  -9.70588235,  -9.58823529,  -9.47058824,
        -9.35294118,  -9.23529412,  -9.11764706,  -9.        ,
        -8.88235294,  -8.76470588,  -8.64705882,  -8.52941176,
        -8.41176471,  -8.29411765,  -8.17647059,  -8.05882353,
        -7.94117647,  -7.82352941,  -7.70588235,  -7.58

# New DBSCAN function

In [244]:
type(event1_pt)

pandas.core.series.Series

In [260]:
def pv_dbscan_single_event(z0:pd.Series, pt:pd.Series, eps=0.08, minPts=2)->list:
    _df = pd.DataFrame({})
    _df['z0'] = z0
    _df['pt'] = pt
    db_clustering = DBSCAN(eps=eps, min_samples=minPts).fit(_df['z0'].values.reshape(-1, 1))
    
    _df['db_labels'] = db_clustering.labels_
    
    #Floor noise points to pt=0
    _df.loc[_df['db_labels']==-1, 'pt'] = 0
    
    pv_label = _df.groupby('db_labels')['pt'].sum().sort_values(ascending=False).index[0]
    
    _df.loc[_df['db_labels']!=pv_label, 'db_labels'] = 0
    _df.loc[_df['db_labels']==pv_label, 'db_labels'] = 1
    
    return _df['db_labels']
    
    
    
    

In [261]:
a = pv_dbscan_single_event(event1_z0, event1_pt)

In [283]:
a

entry  subentry
0      0           0
       1           0
       2           1
       3           0
       4           1
                  ..
       154         0
       155         0
       156         0
       157         0
       158         1
Name: db_labels, Length: 159, dtype: int64

In [286]:
def pv_dbscan2(
    z0: np.array, pt: np.array, eps: float, minPts: int, remove_noise: bool = True
) -> pd.Series:
    "Event-level DBSCAN implementation to return the reconstructed primary vertex"

    _df = pd.DataFrame({})

    _df["z0"] = z0
    _df["pt"] = pt

    db_clustering = DBSCAN(eps=eps, min_samples=minPts).fit(
        _df["z0"].values.reshape(-1, 1)
    )

    _df["db_label"] = db_clustering.labels_

    # Floor noise points to pt=0
    if remove_noise:
        _df.loc[_df["db_label"] == -1, "pt"] = 0

    pv_label = _df.groupby("db_label")["pt"].sum().sort_values(ascending=False).index[0]

    _df.loc[_df["db_label"] != pv_label, "db_label"] = 0
    _df.loc[_df["db_label"] == pv_label, "db_label"] = 1

    return _df["db_label"]


def run_dbscan2(
    df: pd.DataFrame,
    z0_column: str = "trk_z0",
    pt_column: str = "trk_pt",
    eps: float = 0.08,
    minPts: int = 2,
    remove_noise: bool = True,
) -> pd.DataFrame:
    "run dbscan"

    z0_dbscan = df.groupby(level=0).apply(
        lambda x: pv_dbscan2(x[z0_column], x[pt_column], eps, minPts, remove_noise)
    )

    return z0_dbscan

In [266]:
dbscan = run_dbscan2(trk)

In [287]:
dbscan_noise = run_dbscan2(trk, remove_noise = False)

In [271]:
trk['db_label'] =dbscan.values

In [288]:
trk['db_label_noise'] = dbscan_noise.values

In [274]:
trk.groupby(['entry','db_label'])['trk_z0'].mean().reset_index()

entry   db_label
0       0          -0.670527
        1           4.970129
1       0          -0.401582
        1          -3.596191
2       0           2.257422
                      ...   
222973  1          -3.564116
222974  0          -0.670573
        1          -4.368164
222975  0          -0.448482
        1           1.098890
Name: trk_z0, Length: 405787, dtype: float32

In [296]:
mc['pv_MC']

entry   subentry
0       0           5.055163
1       0          -3.608661
2       0          -0.261453
3       0           2.803285
4       0          -1.400667
                      ...   
222971  0          -0.698029
222972  0           0.520282
222973  0          -3.604865
222974  0          -4.483971
222975  0           1.365169
Name: pv_MC, Length: 222976, dtype: float32

In [291]:
def reco_pv_z0(df:pd.DataFrame, group_label:str) -> pd.DataFrame:
    
    df = df[df[group_label]==1]
    
    return df.groupby(['entry', group_label])['trk_z0'].median()
    

In [277]:
res

NameError: name 'res' is not defined

In [292]:
dbscan_z0 = reco_pv_z0(trk, 'db_label')

In [293]:
dbscan_noise_z0 = reco_pv_z0(trk, 'db_label_noise')

In [279]:
from pv_utils import primary_vertex_efficiency

In [294]:
primary_vertex_efficiency(z0_gen=mc['pv_MC'].values, z0_reco=dbscan_z0.values)

(63.36690944316877, 0.20037719447686442, 0.2000296663261878)

In [295]:
primary_vertex_efficiency(z0_gen=mc['pv_MC'].values, z0_reco = dbscan_noise_z0.values)

(59.06375574052812, 0.20443653077657586, 0.20420088797465752)

In [280]:
mc

,,pv_MC
entry,subentry,
0,0,5.055163
1,0,-3.608661
2,0,-0.261453
3,0,2.803285
4,0,-1.400667
...,...,...
222971,0,-0.698029
222972,0,0.520282
222973,0,-3.604865


trk_pt   trk_eta   trk_phi  trk_d0    trk_z0  trk_fake  \
entry  subentry                                                             
0      0         2.623967  1.301414 -0.289234   999.0 -0.878906         2   
       1         3.435026  0.566814 -0.159267   999.0  3.339844         2   
       2         3.774908  1.815973 -0.240375   999.0  4.921875         1   
       3         2.641448 -1.435747 -0.128747   999.0 -0.468750         2   
       4         2.320653  1.837421  0.032604   999.0  4.980469         1   
...                   ...       ...       ...     ...       ...       ...   
222975 211       2.044568 -2.407456 -0.857553   999.0  1.113281         2   
       212       2.491473 -2.141478 -0.616329   999.0  8.847656         2   
       213       2.669393  2.230316 -0.748241   999.0 -0.234375         2   
       214       3.767222 -1.994298 -0.753691   999.0 -4.746094         2   
       215       1.988538 -2.052655 -0.790385   999.0  0.996094         2   

                 is_pv  FH_label DBSCAN_labels  db_label  
entry  subentry                                           
0      0             0         0             0         0  
       1             0         0            -1         0  
       2             1         1             1         1  
       3             0         0             2         0  
       4             1         1             1         1  
...                ...       ...           ...       ...  
222975 211           0         0             3         1  
       212           0         0            19         0  
       213           0         0            22         0  
       214           0         0             9         0  
       215           0         0             3         1  

[40780722 rows x 10 columns]

# Debug test

In [301]:
from pv_utils import run_dbscan as old_scan

In [298]:
test = old_scan(trk)

In [302]:
test_no_noise = old_scan(trk,remove_noise=True)

In [300]:
primary_vertex_efficiency(z0_gen=mc['pv_MC'].values, z0_reco=test['z0'].values)

(70.41385619977038, 0.18993769422462492, 0.18940690773108315)

In [303]:
primary_vertex_efficiency(z0_gen=mc['pv_MC'].values, z0_reco=test_no_noise['z0'].values)

(75.22603329506315, 0.17973611057971084, 0.1790801424430981)